In [2]:
company_list = ['Facebook', 'Nvidia', 'Merck']
years = ['2019', '2018']

filing_type = '10-k'

In [ ]:
curr_ratio_dict = {}
per_change_dict = {}
bvps_dict = {}
eps_dict = {}
week_change_dict = {}
month_change_dict = {}
stock_price_dict = {}

for companies_found in range(len(published_date_dict.keys())):
    
    ticker = list(published_date_dict.keys())[companies_found]
    
    comp_curr_ratio_dict = {}
    comp_per_change_dict = {}
    comp_bvps_dict = {}
    comp_eps_dict = {}
    comp_week_change_dict = {}
    comp_month_change_dict = {}
    comp_stock_price_dict = {}
    
    for date in published_date_dict[ticker]: 
        
        y = date [0:4]
        m = date [5:7]
        d = date [9:10]
        
        stock_date = f'{m}/{d}/{y}'
        
        stock_path = os.path.join('..','Stock_Data','API_Stock_Data',f'{ticker}', f'{ticker}_{y}.csv')
        stock_data_df = pd.read_csv(stock_path)
        
        
        def find_entry():
            for r in c_assets_df.iloc[:, 2]:
                #Need to use date format finder instead
                cref_list.append(int(r[-10:-3]))
        
        #Get the saved 10-K     
        file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{date}.csv')
        df = pd.read_csv(file_path)
        
        #Find the Current Ratio
        c_assets_df = df[df['0'].str.contains('us-gaap:assetscurrent')]
        cref_list = []
        find_entry()
        curr_assets = c_assets_df[c_assets_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        c_liab_df = df[df['0'].str.contains('us-gaap:liabilitiescurrent')]
        cref_list = []
        find_entry()
        curr_liab = c_liab_df[c_liab_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        curr_ratio = (curr_assets / curr_liab).round(2)
        comp_curr_ratio_dict[date] = curr_ratio
        
        #Find Book-Value Per Share
        share_equity_df = df[df['0'].str.contains('us-gaap:stockholdersequity')]
        cref_list = []
        find_entry()
        share_equity = share_equity_df[share_equity_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        shares_out_df = df[df['0'].str.contains('us-gaap:commonstocksharesoutstanding')]
        cref_list = []
        find_entry()
        shares_out = shares_out_df[shares_out_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        bvps = share_equity / shares_out
        comp_bvps_dict[date] = bvps
        
        
        #Find Earnings per Share
        inc_df = df[df['0'].str.contains('us-gaap:comprehensiveincomenetoftax')]
        cref_list = []
        find_entry()
        inc = inc_df[inc_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        eps = inc / shares_out
        comp_eps_dict[date] = eps.round(2)
        
        #Find the pertinent Stock Data
        release_day_data = stock_data_df[stock_data_df['Time'] == date]
        release_day_close = release_day_data['Last']
        
        release_day_index = int(release_day_data.index.values)
        
        prior_day_data = stock_data_df.iloc[release_day_index-1]
        next_day_data = stock_data_df.iloc[release_day_index+1]
        five_day_data = stock_data_df.iloc[release_day_index+4]
        month_day_data = stock_data_df.iloc[release_day_index+19]
        
        prior_day_close = prior_day_data['Last']
        next_day_close = next_day_data['Last']
        five_day_close = five_day_data['Last']
        month_day_close = month_day_data['Last']
        
        comp_stock_price_dict[date] = int(next_day_close)
        
        per_change = ((next_day_close / prior_day_close) - 1) * 100
        comp_per_change_dict[date] = per_change.round(2)
        
        week_change = ((five_day_close / prior_day_close) - 1) * 100
        comp_week_change_dict[date] = week_change.round(2)
        
        month_change = ((month_day_close / prior_day_close) -1) * 100
        comp_month_change_dict[date] = month_change.round(2)
        
    curr_ratio_dict[ticker] = comp_curr_ratio_dict
    per_change_dict[ticker] = comp_per_change_dict
    bvps_dict[ticker] = comp_bvps_dict
    eps_dict[ticker] = comp_eps_dict
    week_change_dict[ticker] = comp_week_change_dict
    month_change_dict[ticker] = comp_month_change_dict
    stock_price_dict[ticker] = comp_stock_price_dict
print("Success")

In [3]:
# Imports
import pandas as pd
import os
import bs4
import pickle
import requests
from bs4 import BeautifulSoup
import sys
import matplotlib.pyplot as plt


# Defining Functions

# Checking Companies
def company_check(company_list):
    key_path = os.path.join('..','Financials', 'Ticker_and_CIK', 'Key.csv')
    key_df = pd.read_csv(key_path)
    key_dict = {}

    for query in range(len(company_list)):
        try:
            company_df = key_df[key_df['Company'].str.contains(company_list[query].title())]
            key_dict[company_df.iloc[0, 1]] = [company_df.iloc[0, 0], company_df.iloc[0, 2]]
        except IndexError:
            print(f"{company_list[query]} could not be found")
    if key_dict == {}:
        print ("Whoops, none of your companies were on the S&P 500. Please try again!")
    else:
        print (f"Success! We were able to find {len(key_dict.keys())} of your queries in the S&P 500")
        print (f"We found: {list(key_dict.keys())} in the S&P 500")
        return key_dict

    
# Obtaining Financials from the SEC
def fetch_document_links(cells, year):
    doc_link = 'https://www.sec.gov' + cells[1].a['href']
    # Obtain HTML for document page
    doc_resp = requests.get(doc_link)
    doc_str = doc_resp.text
    # Find the XBRL link
    soup_doc = BeautifulSoup(doc_str, 'html.parser')
    table_tag_doc = soup_doc.find('table', class_='tableFile', summary='Data Files')
    rows_doc = table_tag_doc.find_all('tr')
    for row_doc in rows_doc:
        cells_doc = row_doc.find_all('td')
        if len(cells_doc) > 3:
                if 'XML' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href']
                elif 'INS' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href'] 
                
def fetch_financial_data(ticker, tag_list, as_of_dates, financial_dict):
    row_num = 0
    for tag in tag_list:
        if 'schemaref' in tag.name:
            ignored_href_index = len(ticker) + 1
            as_of_date = tag['xlink:href'][ignored_href_index:(ignored_href_index+8)]
            as_of_dates.append(as_of_date)
            
        if tag.name.find('us-gaap:') != -1:
            try: 
                if float(tag.text) > 0:
                    i = [f'{tag.name}', tag.text, tag['contextref']]
                    financial_dict[f'{row_num}'] = i
                    row_num = row_num + 1
            except ValueError:
                pass

            
# Unmodified Base Code found at https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
def financial_scraper(key_dict, filing_type, published_date_dict, as_of_date_dict, agg_financial_dict):
    #Looping through Companies in List
    for companies_found in range(len(key_dict.keys())):
        # Obtaining Ticker Symbol
        company_name = list(key_dict.keys())[companies_found]
        ticker = key_dict.get(company_name)[0]

        cik = key_dict.get(company_name)[1]
        dateb = f'{years[0]}-12-31'

        # Obtain HTML for search page
        base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
        edgar_resp = requests.get(base_url.format(cik, filing_type, dateb))
        edgar_str = edgar_resp.text

        doc_links = []
        published_dates = []
        as_of_dates = []

        # Find the document link
        soup = BeautifulSoup(edgar_str, 'html.parser')
        table_tag = soup.find('table', class_='tableFile2')
        rows = table_tag.find_all('tr')
        
        all_years_dict = {}
        
        for year in years:
            for row in rows:
                cells = row.find_all('td')
                if len(cells) > 3:
                    if year in cells[3].text:
                        published_date = cells[3].text
                        published_dates.append(published_date)

                        xbrl_link = fetch_document_links(cells, year)
                        # Obtain XBRL text from document
                        xbrl_resp = requests.get(xbrl_link)
                        xbrl_str = xbrl_resp.text

                        # Find and print stockholder's equity
                        soup = BeautifulSoup(xbrl_str, 'lxml')
                        tag_list = soup.find_all()

                        financial_dict = {}

                        fetch_financial_data(ticker, tag_list, as_of_dates, financial_dict)
                 
                        all_years_dict[f'{published_date}'] = financial_dict
                    
        published_date_dict[ticker] = published_dates
        as_of_date_dict[ticker] = as_of_dates
        agg_financial_dict[ticker] = all_years_dict
        
        
        
# Finding Entries already Saved as Variable
def find_entries(as_of_date_dict, published_date_dict, agg_financial_dict, journal_entry_name_list):
    all_financial_dict = {}
    
    for companies_found in range(len(published_date_dict.keys())):
        ticker = list(published_date_dict.keys())[companies_found]

        comp_curr_ratio_dict = {}
        comp_per_change_dict = {}
        comp_bvps_dict = {}
        comp_eps_dict = {}
        comp_week_change_dict = {}
        comp_month_change_dict = {}
        comp_stock_price_dict = {}
        
        comp_financials_dict = {}

        for date_index in range(len(as_of_date_dict[ticker])):
            df = pd.DataFrame.from_dict(agg_financial_dict[ticker][(published_date_dict[ticker][date_index])], orient = 'index')
            
            date_index_financials_df = pd.DataFrame()
            
            for entry in journal_entry_name_list:
                c_assets_df = df[df[0] == entry]
                example = find_entry(as_of_date_dict, ticker, date_index, c_assets_df, entry)
                date_index_financials_df = date_index_financials_df.append(example)    
           
            comp_financials_dict[published_date_dict[ticker][date_index]] = date_index_financials_df.iloc[:, 0:2].to_dict()
        all_financial_dict[ticker] = comp_financials_dict
    return all_financial_dict
            
def find_entry(as_of_date_dict, ticker, date_index, c_assets_df, entry):
    
    test_date = as_of_date_dict[ticker][date_index]
    y = test_date [0:4]
    m = test_date [4:6]
    d = test_date [6:8]
    
    year_check = int(y) + 1
    
    quarter_list = ['Q4', 'Q3', "Q2", "Q1"]
    context_ref = c_assets_df[2]
    test_df = pd.DataFrame()

    if len(context_ref) == 1:
        # Only one entry
        test_df = test_df.append(c_assets_df)
        #print('Only 1 entry')
    elif len(context_ref) == 0:
        print(f'Error, There are no Entries for that {ticker}{test_date}{entry}')
        
    else:
        while year_check >= (int(y)-1):
            year_check_df = c_assets_df[context_ref.str.contains(f'{year_check}')]
            
            if len(year_check_df) == 1:
                # Only 1 entry for the year
                test_df = test_df.append(year_check_df)
                #print('Only 1 entry of that year')
                break
            else: 
                date_formats = [f'{year_check}{m}{d}', f'{year_check}-{m}-{d}', f'{m.lstrip("0")}_{d.lstrip("0")}_{year_check}']

                for item in date_formats:
                    # Check through yyyymmdd, yyyy-mm-dd, and mm_dd_yyyy
                    if len(c_assets_df[context_ref.str.contains(f'{item}')]) == 1:
                        # Only 1 entry
                        test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{item}')])
                        #print(f'Context Ref uses {item} format')
                        break
                    elif len(c_assets_df[context_ref.str.contains(f'{item}')]) > 1:
                        # Multiple Entries (usually in case of stockholder equity)
                        stock_holder_df = c_assets_df[context_ref.str.contains(f'{item}')]
                        stock_holder_df[1] = stock_holder_df[1].map(int)
                        # Take Max Value given
                        test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[0]
                        break
                else:
                    for quarter in quarter_list:
                        # Check through q#yyyy
                        if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) > 0:
                            if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) == 1:
                                test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')])
                                #print(f'Context Ref uses {quarter}{year_check} format')
                                break
                            elif len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) > 1:
                                # Multiple Entries (usually in case of stockholder equity)
                                stock_holder_df = c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]
                                stock_holder_df[1] = stock_holder_df[1].map(int)
                                test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[0]
                                break
                        elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) > 0:
                            if len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) == 1:
                                test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')])
                                #print(f'Context Ref uses {year_check}{quarter} format')
                                break
                            elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) > 1:
                                # Multiple Entries (usually in case of stockholder equity)
                                stock_holder_df = c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]
                                stock_holder_df[1] = stock_holder_df[1].map(int)
                                test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[0]
                                break
                if len(test_df) == 0:
                    year_check = year_check - 1
                else:
                    year_check = int(y) - 2
    return test_df

            
# Defining Date Dictionaries
published_date_dict = {}
as_of_date_dict = {}
agg_financial_dict = {}

journal_entry_name_list = ['us-gaap:cashandcashequivalentsatcarryingvalue', 'us-gaap:assetscurrent', 'us-gaap:assets',\
                                       'us-gaap:liabilitiescurrent', 'us-gaap:liabilities', 'us-gaap:stockholdersequity', \
                                       'us-gaap:cashandcashequivalentsatcarryingvalue']

# Running Functions
key_dict = company_check(company_list)
financial_scraper(key_dict, filing_type, published_date_dict, as_of_date_dict, agg_financial_dict)
html_table_dict = find_entries(as_of_date_dict, published_date_dict, agg_financial_dict, journal_entry_name_list)
print(html_table_dict)

Success! We were able to find 3 of your queries in the S&P 500
We found: ['Facebook, Inc.', 'Nvidia Corporation', 'Merck & Co.'] in the S&P 500


C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

Error, There are no Entries for that MRK20171231us-gaap:liabilities
{'FB': {'2019-01-31': {0: {'91': 'us-gaap:cashandcashequivalentsatcarryingvalue', '52': 'us-gaap:assetscurrent', '50': 'us-gaap:assets', '376': 'us-gaap:liabilitiescurrent', '372': 'us-gaap:liabilities', '600': 'us-gaap:stockholdersequity'}, 1: {'91': 10019000000, '52': '50480000000', '50': '97334000000', '376': '7017000000', '372': '13207000000', '600': 84127000000}}, '2018-02-01': {0: {'86': 'us-gaap:cashandcashequivalentsatcarryingvalue', '58': 'us-gaap:assetscurrent', '56': 'us-gaap:assets', '355': 'us-gaap:liabilitiescurrent', '351': 'us-gaap:liabilities', '584': 'us-gaap:stockholdersequity'}, 1: {'86': 8079000000, '58': '48563000000', '56': '84524000000', '355': '3760000000', '351': '10177000000', '584': 74347000000}}}, 'NVDA': {'2019-02-21': {0: {'131': 'us-gaap:cashandcashequivalentsatcarryingvalue', '54': 'us-gaap:assetscurrent', '52': 'us-gaap:assets', '453': 'us-gaap:liabilitiescurrent', '446': 'us-gaap:liab

C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [4]:
curr_ratio_dict = {}
per_change_dict = {}
bvps_dict = {}
eps_dict = {}
week_change_dict = {}
month_change_dict = {}
stock_price_dict = {}

for companies_found in range(len(published_date_dict.keys())):
    
    ticker = list(published_date_dict.keys())[companies_found]
    
    comp_curr_ratio_dict = {}
    comp_per_change_dict = {}
    comp_bvps_dict = {}
    comp_eps_dict = {}
    comp_week_change_dict = {}
    comp_month_change_dict = {}
    comp_stock_price_dict = {}
    
    for date in published_date_dict[ticker]: 
        
        y = date [0:4]
        m = date [5:7]
        d = date [9:10]
        
        stock_date = f'{m}/{d}/{y}'
        
        stock_path = os.path.join('..','Stock_Data','API_Stock_Data',f'{ticker}', f'{ticker}_{y}.csv')
        stock_data_df = pd.read_csv(stock_path)
        
        
        def find_entry():
            for r in c_assets_df.iloc[:, 2]:
                #Need to use date format finder instead
                cref_list.append(int(r[-10:-3]))
        
        #Get the saved 10-K     
        file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{date}.csv')
        df = pd.read_csv(file_path)
        
        #Find the Current Ratio
        c_assets_df = df[df['0'].str.contains('us-gaap:assetscurrent')]
        cref_list = []
        find_entry()
        curr_assets = c_assets_df[c_assets_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        c_liab_df = df[df['0'].str.contains('us-gaap:liabilitiescurrent')]
        cref_list = []
        find_entry()
        curr_liab = c_liab_df[c_liab_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        curr_ratio = (curr_assets / curr_liab).round(2)
        comp_curr_ratio_dict[date] = curr_ratio
        
        #Find Book-Value Per Share
        share_equity_df = df[df['0'].str.contains('us-gaap:stockholdersequity')]
        cref_list = []
        find_entry()
        share_equity = share_equity_df[share_equity_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        shares_out_df = df[df['0'].str.contains('us-gaap:commonstocksharesoutstanding')]
        cref_list = []
        find_entry()
        shares_out = shares_out_df[shares_out_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        bvps = share_equity / shares_out
        comp_bvps_dict[date] = bvps
        
        
        #Find Earnings per Share
        inc_df = df[df['0'].str.contains('us-gaap:comprehensiveincomenetoftax')]
        cref_list = []
        find_entry()
        inc = inc_df[inc_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        eps = inc / shares_out
        comp_eps_dict[date] = eps.round(2)
        
        #Find the pertinent Stock Data
        release_day_data = stock_data_df[stock_data_df['Time'] == date]
        release_day_close = release_day_data['Last']
        
        release_day_index = int(release_day_data.index.values)
        print(release_day_index)
        
        prior_day_data = stock_data_df.iloc[release_day_index-1]
        next_day_data = stock_data_df.iloc[release_day_index+1]
        five_day_data = stock_data_df.iloc[release_day_index+4]
        month_day_data = stock_data_df.iloc[release_day_index+19]
        
        prior_day_close = prior_day_data['Last']
        next_day_close = next_day_data['Last']
        five_day_close = five_day_data['Last']
        month_day_close = month_day_data['Last']
        
        comp_stock_price_dict[date] = int(next_day_close)
        
        per_change = ((next_day_close / prior_day_close) - 1) * 100
        comp_per_change_dict[date] = per_change.round(2)
        
        week_change = ((five_day_close / prior_day_close) - 1) * 100
        comp_week_change_dict[date] = week_change.round(2)
        
        month_change = ((month_day_close / prior_day_close) -1) * 100
        comp_month_change_dict[date] = month_change.round(2)
        
    curr_ratio_dict[ticker] = comp_curr_ratio_dict
    per_change_dict[ticker] = comp_per_change_dict
    bvps_dict[ticker] = comp_bvps_dict
    eps_dict[ticker] = comp_eps_dict
    week_change_dict[ticker] = comp_week_change_dict
    month_change_dict[ticker] = comp_month_change_dict
    stock_price_dict[ticker] = comp_stock_price_dict
print("Success")

FileNotFoundError: [Errno 2] No such file or directory: '..\\Stock_Data\\API_Stock_Data\\FB\\FB_2019.csv'